## Get Exome

In [1]:
import pybedtools
from gtfparse import read_gtf
from ftplib import FTP

In [2]:
#file_genome_sequence = './data/Homo_sapiens.GRCh38.104.dna_rm.primary_assembly.fa'
#file_gene_annotation = './data/Homo_sapiens.GRCh38.104.gtf'

In [3]:
def download_genome_fasta(dir_data, release='104'):
    ftp = FTP('ftp.ensembl.org')
    ftp.login() 

    # move to genome directory
    directory = 'pub/release-{}/fasta/homo_sapiens/dna'.format(release)
    ftp.cwd(directory)

    allfiles = ftp.nlst()

    for file in allfiles:
        if 'dna_rm.primary_assembly' in file:
            file_genome_sequence = file
            ftp.retrbinary('RETR ' + file, open(os.path.join(dir_data, file), 'wb').write)
            print(file_genome_sequence)

    ftp.quit()

    return dir_data+file_genome_sequence

In [11]:
def download_gene_gtf(dir_data, release='104'):
    ftp = FTP('ftp.ensembl.org')
    ftp.login() 

    # move to annotation directory
    directory = 'pub/release-{}/gtf/homo_sapiens/'.format(release)
    ftp.cwd(directory)

    allfiles = ftp.nlst()

    for file in allfiles:
        if '{}.gtf.gz'.format(release) in file:
            file_gene_annotation = file
            ftp.retrbinary('RETR ' + file, open(os.path.join(dir_data, file), 'wb').write)
            print(file_gene_annotation)

    ftp.quit()

    return dir_data+file_gene_annotation


In [5]:
def get_exons_annotation(file_gene_annotation):

    file_exon_annotation = '{}.exons.gtf'.format(file_gene_annotation.split('.gtf')[0])
    
    # extract exons from gene annotation file
    gene_annotation = read_gtf(file_gene_annotation)
    
    exon_annotation = gene_annotation[gene_annotation["feature"] == "exon"]
    #exon_annotation = exon_annotation[exon_annotation["transcript_biotype"] == "protein_coding"]
    exon_annotation[['seqname','source','exon_id','start','end','score','strand','frame','gene_id']].to_csv(file_exon_annotation, sep='\t', header=False, index = False)
    
    return file_exon_annotation

In [6]:
def get_exons_fasta(file_exon_annotation, file_genome_sequence):
    
    file_exon_sequence = '{}.fa'.format(file_exon_annotation.split('.gtf')[0])
    
    # get sequence for exons
    exon_annotation = pybedtools.BedTool(file_exon_annotation)
    genome_sequence = pybedtools.BedTool(file_genome_sequence)

    exon_annotation = exon_annotation.sequence(fi=genome_sequence, s=True, name=True)
    exon_annotation.save_seqs(file_exon_sequence)
    
    return file_exon_sequence

In [8]:
file_genome_sequence = download_genome_fasta('./data/')
cmd = "gunzip {}".format(file_gene_annotation)
os.system(cmd)
file_genome_sequence = file_gene_annotation.split('.gz')
file_genome_sequence

Homo_sapiens.GRCh38.dna_rm.primary_assembly.fa.gz


In [13]:
file_gene_annotation = download_gene_gtf('./data/')
cmd = "gunzip {}".format(file_gene_annotation)
os.system(cmd)
file_gene_annotation = file_gene_annotation.split('.gz')
file_gene_annotation

Homo_sapiens.GRCh38.104.gtf.gz


In [ ]:
# get exon annotation
file_exon_annotation = get_exons_annotation(file_gene_annotation)

In [ ]:
# get exon sequences
file_exon_sequence = get_exons_fasta(file_exon_annotation, file_genome_sequence)

## Get potential probes

In [ ]:
import os
import copy
import pickle
import pandas as pd

from gtfparse import read_gtf

from Bio import SeqIO
from Bio.SeqUtils import GC
from Bio.SeqUtils import MeltingTemp as mt

In [ ]:
file_exon_sequence = './data/Homo_sapiens.GRCh38.104.exons.fa'
file_gene_annotation = './data/Homo_sapiens.GRCh38.104.gtf'

In [ ]:
GC_content_min = 40
GC_content_max = 60
Tm_min = 68
Tm_max = 75
probe_length = 45
min_probes_per_gene = 4

In [ ]:
GC_content_min = float(GC_content_min)
GC_content_max = float(GC_content_max)
Tm_min = float(Tm_min)
Tm_max = float(Tm_max)

In [ ]:
gene_annotation = read_gtf(file_gene_annotation)
exon_annotation = gene_annotation[gene_annotation["feature"] == "exon"]

mapping_exon_to_gene = pd.Series(exon_annotation.gene_id.values,index=exon_annotation.exon_id).to_dict()
mapping_gene_to_exon = pd.Series(exon_annotation.exon_id.values,index=exon_annotation.gene_id).to_dict()

pickle.dump(mapping_exon_to_gene, open('./data/mapping_exon_to_gene.pkl','wb'))
pickle.dump(mapping_gene_to_exon, open('./data/mapping_gene_to_exon.pkl','wb'))


In [ ]:
mapping_exon_to_gene = pickle.load(open('./data/mapping_exon_to_gene.pkl','r'))
mapping_gene_to_exon = pickle.load(open('./data/mapping_gene_to_exon.pkl','r'))

In [ ]:
number_exons = 0
number_probes = 0

mapping_gene_to_probes = {}

for exon in SeqIO.parse(file_exon_sequence, "fasta"):

    number_exons += 1
    exon_id = exon.id.split('::')[0]
    gene_id = mapping_exon_to_gene[exon_id]

    exon_sequence = exon.seq
    if len(exon_sequence) > probe_length:
        probes_of_exon = set([exon_sequence[i:i+probe_length] for i in range(len(exon_sequence)-(probe_length-1)) if ('N' not in exon_sequence[i:i+probe_length])])
        number_probes += len(probes_of_exon)
        for probe in probes_of_exon:
            gc_content = GC(probe)
            if (GC_content_min < gc_content < GC_content_max):
                Tm = mt.Tm_NN(probe)
                if (Tm_min < Tm < Tm_max):
                    if gene_id in mapping_gene_to_probes:
                        if probe in mapping_gene_to_probes[gene_id]:
                            mapping_gene_to_probes[gene_id][probe].append(exon_id)
                        else:
                            mapping_gene_to_probes[gene_id][probe] = [exon_id]
                    else:
                        mapping_gene_to_probes[gene_id] = {probe: [exon_id]}
                
    if number_exons % 10000 == 0:
        print(number_exons)

pickle.dump(mapping_gene_to_probes, open('./data/mapping_gene_to_probes.pkl','wb'))       
        

In [ ]:
mapping_gene_to_probes = pickle.load(open('./data/mapping_gene_to_probes.pkl','r'))

In [ ]:
print('exons: {}'.format(number_exons))
print('probes: {}'.format(number_probes))

In [ ]:
probes_unique = {}
probes_non_unique = {}

mapping_gene_to_probes_filtered = copy.deepcopy(mapping_gene_to_probes)

for gene_id, mapping_probe_to_exon in mapping_gene_to_probes.items():
    for probe, exon_id in mapping_probe_to_exon.items():
        if probe in probes_non_unique:
            probes_non_unique[probe] += 1
            mapping_gene_to_probes_filtered[gene_id].pop(probe)
        elif probe in probes_unique:
            mapping_gene_to_probes_filtered[probes_unique[probe][1]].pop(probe)
            mapping_gene_to_probes_filtered[gene_id].pop(probe)
            probes_unique.pop(probe)
            probes_non_unique[probe] = 2
        else:
            probes_unique[probe] = [exon_id, gene_id]

pickle.dump(mapping_gene_to_probes_filtered, open('./data/mapping_gene_to_probes_filtered.pkl','wb'))  


In [ ]:
mapping_gene_to_probes_filtered = pickle.load(open('./data/mapping_gene_to_probes_filtered.pkl','r'))

In [ ]:
print('unique probes {}'.format(len(probes_unique)))

removed_genes = []

for gene_id, mapping_probe_to_exon in mapping_gene_to_probes_filtered.items():
    if len(mapping_probe_to_exon) < min_probes_per_gene:
        removed_genes.append(gene_id)

removed_genes

## Get potential probes - old version

In [ ]:
probes_unique = {}
probes_non_unique = {}


for exon in SeqIO.parse(file_exon_sequence, "fasta"):
    #print(exon.id)
    #print(exon.seq)
    number_exons += 1
    if len(exon.seq) > probe_length:
        for i in range(0, len(exon.seq)-(probe_length-1), 1):
            number_probes += 1
            probe = exon.seq[i:i+probe_length]
            if probe in probes_non_unique:
                probes_non_unique[probe] += 1
            elif probe in probes_unique:
                probes_unique.pop(probe)
                probes_non_unique[probe] = 2
            else:
                if not 'N' in probe:
                    gc_content = GC(probe)
                    if (gc_content > GC_content_min) & (gc_content < GC_content_max):
                        Tm = mt.Tm_NN(probe)
                        if (Tm > Tm_min) & (Tm < Tm_max):
                            probes_unique[probe] = exon.id

In [ ]:
non_unique = 0
for key, value in probes_non_unique.items():
    non_unique += value


print('exons: {}'.format(number_exons))
print('probes: {}'.format(number_probes))
print('unique probes {}'.format(len(probes_unique)))
print('non unique probes {}'.format(non_unique))

## Bowtie

In [ ]:
file_bowtie_index = file_exon_sequence.split('.fa')[0]
file_exon_sequence = './data/Homo_sapiens.GRCh38.104.exons.fa'

In [ ]:

threads = 4
cmd = 'bowtie-build --threads {} {} {}'.format(file_exon_sequence, file_bowtie_index)
os.system(cmd)

In [ ]:

#cmd = 'bowtie --threads {} -c {} {}'.format(threads, file_bowtie_index, str(probe))
#alignment = sp.check_output(cmd, shell=True)
#if not alignment.decode("utf-8"):
    #print(probe)
    #number_probes_pass_bowtie_filter += 1

In [ ]:
cmd = 'bowtie -c {} {}'.format(file_bowtie_index, 'TTCTACAACGACGTGGTCAGCTCCAAGCCGTGCAAGCCCT')
output1 = sp.check_output(cmd, shell=True)
output1 = output1.decode("utf-8")

In [ ]:
cmd = 'bowtie -c {} {}'.format(file_bowtie_index, 'GAAGAAGGGAGAGAAAGCTCCCTCCTGTGTGTCNNNNNNN')
output2 = sp.check_output(cmd, shell=True)
output2 = output2.decode("utf-8")

In [ ]:
if output1:
    print(output1)
    print('not empty')

In [ ]:
if not output2:
    print(output2)
    print('empty')